<a href="https://colab.research.google.com/github/yuniarachmaa/TUGAS-MATEMATIKA-DISKRIT-A/blob/main/MATDIS_TUGAS_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Yuniar Rachmawati

NPM : 24083010106

Mata Kuliah : Matematika Diskrit (A)

# TUGAS MATEMATIKA DISKRIT 1

In [ ]:
import pandas as pd
import numpy as np
from numpy.linalg import norm

# 1. Baca data Excel
file_path = "DATA MATDIS A.xlsx"
df = pd.read_excel(file_path)

# 2. Ambil kolom C–F sebagai kriteria
C = df.iloc[:, 2:6]

# --- Perbaikan data ---
# Isi NaN dengan mean per kolom
C = C.fillna(C.mean())

# Hapus kolom konstan (supaya korelasi tidak NaN)
C = C.loc[:, C.nunique() > 1]

# Simpan kembali ke dataframe bersih
df_clean = df.copy()
df_clean[C.columns] = C

# Ubah ke numpy array
C = C.to_numpy()

# 3. Tentukan bobot prioritas (matrix P)
P = np.array([0.2, 0.3, 0.4, 0.4])
P = P[:C.shape[1]]   # sesuaikan dengan jumlah kolom C

# 4. Hitung skor total (C × P)
scores = C @ P
df_clean["Skor_Total"] = scores

# Ambil 4 mahasiswa terbaik
top4 = df_clean.nlargest(4, "Skor_Total")

print("Empat mahasiswa terbaik berdasarkan skor total:")
print(top4[["NAMA", "Skor_Total"]])

# 5. Matriks kovarians & korelasi antar kriteria
cov_mat = np.cov(C, rowvar=False)
corr_mat = np.corrcoef(C, rowvar=False)

print("\nMatriks Kovarians:")
print(cov_mat)

print("\nMatriks Korelasi:")
print(corr_mat)

# 6. Cosine similarity antar mahasiswa (handle pembagian 0)
norms = norm(C, axis=1)
norms[norms == 0] = 1e-10   # cegah pembagian 0
cosine_sim = (C @ C.T) / (norms[:, None] * norms)
cosine_df = pd.DataFrame(cosine_sim, index=df["NAMA"], columns=df["NAMA"])

print("\nCosine Similarity (5x5 pertama):")
print(cosine_df.iloc[:5, :5])

# 7. Simpan hasil analisis ke Excel baru
with pd.ExcelWriter("Hasil_Analisis.xlsx") as writer:
    df_clean.to_excel(writer, sheet_name="Skor Mahasiswa", index=False)
    pd.DataFrame(cov_mat).to_excel(writer, sheet_name="Kovarians", index=False)
    pd.DataFrame(corr_mat).to_excel(writer, sheet_name="Korelasi", index=False)
    cosine_df.to_excel(writer, sheet_name="CosineSimilarity")

Empat mahasiswa terbaik berdasarkan skor total:
                               NAMA  Skor_Total
8   RAVEENA AYU DESEMBER SURYOPUTRI        1.25
12             AMELIA RIZQYNA PUTRI        1.25
6             Mohammad Alimun Hakim        1.21
36              Indra Maulana R F Y        1.21

Matriks Kovarians:
[[0.00232742 0.00015779 0.00027613 0.0004931 ]
 [0.00015779 0.00092045 0.00186719 0.00189349]
 [0.00027613 0.00186719 0.00403682 0.00408284]
 [0.0004931  0.00189349 0.00408284 0.0043787 ]]

Matriks Korelasi:
[[1.         0.10780666 0.09008728 0.15446231]
 [0.10780666 1.         0.96865726 0.94317299]
 [0.09008728 0.96865726 1.         0.97111419]
 [0.15446231 0.94317299 0.97111419 1.        ]]

Cosine Similarity (5x5 pertama):
NAMA                     MADINA HEDY ALIFANTI  MARIA DWI KURNIASIH  \
NAMA                                                                 
MADINA HEDY ALIFANTI                 1.000000             1.000000   
MARIA DWI KURNIASIH                  1.000000      

* ## Import Libraries dan Data Loading
Bagian awal mengimpor pandas untuk manipulasi DataFrame, numpy untuk operasi array numerik, dan norm dari numpy.linalg untuk kalkulasi normalisasi vektor. Data mahasiswa dibaca dari file Excel menggunakan `pd.read_excel()` dan disimpan dalam DataFrame untuk preprocessing selanjutnya.

* ## Data Preprocessing dan Cleaning  
Ekstraksi data dilakukan dari kolom C hingga akhir (mengabaikan kolom identitas), kemudian missing values ditangani dengan mean imputation menggunakan `fillna(df.mean())`. DataFrame dibersihkan dan nama kolom dikonversi ke uppercase untuk konsistensi, lalu dikonversi ke numpy array untuk optimasi komputasi numerik.

* ## Student Ranking Algorithm
Implementasi ranking menggunakan sum agregat semua kriteria penilaian per mahasiswa, kemudian `np.argmax()` mengidentifikasi indeks dengan skor tertinggi. Output menunjukkan empat mahasiswa dengan skor optimal identik (1.25): RAVEENA AYU, AMELIA RIZQYNA, Mohammad Alimun, dan Indra Maulana.

* ## Correlation Matrix Analysis
Dua matriks dihitung : kovarians dengan `np.cov()` untuk mengukur variabilitas bersama, dan korelasi dengan `np.corrcoef()` untuk hubungan linear standarisasi. Hasil menunjukkan spektrum korelasi dari negatif hingga positif, mengindikasikan heterogenitas pola akademik antar mahasiswa.

* ## Cosine Similarity Computation
Algoritma menormalisasi setiap vektor mahasiswa menggunakan L2 norm, kemudian menghitung dot product untuk menghasilkan similarity matrix. Cosine similarity mengukur sudut antar vektor (1=identik, 0=ortogonal, -1=berlawanan). Hasil dalam DataFrame menunjukkan beberapa mahasiswa memiliki similarity tinggi seperti MADINA HEDY ALIFANTI, mengindikasikan profil akademik serupa.